In [4]:
pip install requests

  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached urllib3-1.26.11-py2.py3-none-any.whl (139 kB)
  Using cached certifi-2022.6.15-py3-none-any.whl (160 kB)
  Using cached charset_normalizer-2.1.0-py3-none-any.whl (39 kB)
  Using cached idna-3.3-py3-none-any.whl (61 kB)

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:

import pandas as pd
import requests
from bs4 import BeautifulSoup

url = "https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168#.YvXRyOzMI0Q"

# 1. Download the web page
page = requests.get(url)

# 2. Parse the page using bs
soup = BeautifulSoup(page.content, "html.parser")

# 3. Find element with id "seven-day-forecast"
seven_day_forecast = soup.find(id="seven-day-forecast")

# 4. Get forecast for each of 7 periods
# 5. Extract the data for each forecast item
# 5.1 Get the period tags & extract the content
period_tags = seven_day_forecast.select(".tombstone-container .period-name")
periods = [p.get_text(strip=True) for p in period_tags]

# 5.2 Get the short-desc tags & extract the content
short_desc_tags = seven_day_forecast.select(".tombstone-container .short-desc")
# Use of separator to replace any nested tags with space, such as <br/>
short_descs = [sd.get_text(separator=" ", strip=True) for sd in short_desc_tags]

# 5.3 Get the temp tags & extract the content
temp_tags = seven_day_forecast.select(".tombstone-container .temp")
temps = [t.get_text(strip=True) for t in temp_tags]

# 5.4 Get the full desc tags (in img tags) & extract the content
img_tags = seven_day_forecast.select(".tombstone-container img")
descs = [d.get("title") for d in img_tags]  # The desc is in the attr. title of img tag

# 6. Construct Panda dataframe (tabular object)
weather = pd.DataFrame(
    {"period": periods, "short_desc": short_descs, "temp": temps, "desc": descs}
)

# print(weather.to_string())
# more options can be specified also
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     print(weather)

# display(weather)

# 7. Analysis
# 7.1 Get all temp nums as integers
# (?Pd+) means find any matching group (named or unaming group) that satisfies regex d+
# https://stackoverflow.com/questions/7988942/what-does-this-django-regular-expression-mean-p
# temp_nums = weather["temp"].str.extract('(?Pd+)', expand=False)
# weather["temp_num"] = temp_nums.astype('int')
# temp_nums

temp_nums = weather["temp"].str.extract(r"(\d+)", expand=False)
ts = temp_nums.astype("int")
weather["temp_nums"] = ts
print(weather["temp_nums"].mean())

is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night

display(weather)

64.88888888888889


,period,short_desc,temp,desc,temp_nums,is_night
0,Tonight,Mostly Clear and Breezy,Low: 59 °F,"Tonight: Mostly clear, with a low around 59. B...",59,True
1,Friday,Mostly Sunny then Sunny and Breezy,High: 71 °F,"Friday: Sunny, with a high near 71. Breezy, wi...",71,False
2,FridayNight,Clear and Breezy then Mostly Clear,Low: 58 °F,"Friday Night: Mostly clear, with a low around ...",58,True
3,Saturday,Sunny then Sunny and Breezy,High: 73 °F,"Saturday: Sunny, with a high near 73. Breezy, ...",73,False
4,SaturdayNight,Mostly Clear and Breezy then Mostly Clear,Low: 59 °F,"Saturday Night: Mostly clear, with a low aroun...",59,True
5,Sunday,Sunny,High: 72 °F,"Sunday: Sunny, with a high near 72.",72,False
6,SundayNight,Mostly Clear and Breezy then Mostly Clear,Low: 59 °F,"Sunday Night: Mostly clear, with a low around ...",59,True
7,Monday,Sunny,High: 74 °F,"Monday: Sunny, with a high near 74.",74,False
8,MondayNight,Mostly Clear,Low: 59 °F,"Monday Night: Mostly clear, with a low around 59.",59,True
